# Langchain Integrations of Vector Stores

## Retrieval Options

### Maximal Marginal Relevance (MMR)

Maximal Marginal Relevance (MMR) is a method used in information retrieval and text summarization to identify and select diverse and relevant documents or parts of documents from a larger set of information. The goal of MMR is to balance two aspects:

1. **Relevance**: How well the selected documents or snippets match the query or the main theme of the text.
2. **Diversity**: How different each selected document or snippet is from others to avoid redundancy.

#### Mathematical Formulation:

The MMR score for a candidate item $D_i$ (a document or snippet) is calculated relative to a query $Q$ and a set of already-selected items $S$ using the following formula:

$$\text{MMR} = \lambda \cdot \text{Relevance}(D_i, Q) - (1 - \lambda) \cdot \max_{D_j \in S} \text{Similarity}(D_i, D_j)$$

- $\text{Relevance}(D_i, Q)$ measures how relevant document $D_i$ is to the query $Q$.
- $\text{Similarity}(D_i, D_j)$ measures how similar document $D_i$ is to another already-selected document $D_j$.
- $\lambda$ is a parameter that controls the trade-off between relevance and diversity. It ranges from 0 to 1, with higher values giving more weight to relevance and lower values emphasizing diversity.

#### Intuition Behind MMR:

- **When $\lambda$ is high**: The algorithm focuses more on relevance, preferring items that are highly related to the query, even if they are similar to each other.
- **When $\lambda$ is low**: The algorithm emphasizes diversity, selecting items that are different from each other, even if they are less relevant to the query.

This balancing act ensures that the final set of retrieved documents or snippets covers different aspects or viewpoints related to the query, providing a more comprehensive and diverse understanding of the topic, which is particularly valuable in contexts like search engine results, news aggregation, or literature reviews.


## Chroma

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

Chroma runs in various modes. See below for examples of each integrated with LangChain.

- in-memory - in a python script or jupyter notebook
- in-memory with persistance - in a script or notebook and save/load to disk
- in a docker container - as a server running your local machine or in the cloud

Like any other database, you can:

- `.add`
- `.get`
- `.update`
- `.upsert`
- `.delete`
- `.peek`
and `.query` runs the similarity search.

View full docs at [docs](https://docs.trychroma.com/reference/Collection). To access these methods directly, you can do `._collection.method()`



In [ ]:
pip install chromadb

In [ ]:
pip install langchain

In [ ]:
pip install sentence-transformers

In [4]:
# import
from langchain.document_loaders import TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

### Basic Example

In this basic example, we take the most recent State of the Union Address, split it into chunks, embed it using an open-source embedding model, load it into Chroma, and then query it.

In [12]:
# create a fake txt file
with open('some_file.txt', 'w') as f:
  f.write(
      """
        My mother's name is Jenny. \n

        My uncle's name is Tim. \n

        We live in Avatar City, Dumbo Street, United States of Dickens.
      """
  )

In [13]:
# load the document and split it into chunks
loader = TextLoader("some_file.txt")
documents = loader.load()

In [14]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [15]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [16]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

In [19]:
# query it
query = "What the mother's name?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

My mother's name is Jenny. My uncle's name is Tim. We live in Avatar City, Dumbo Street, United States of Dickens.


In [20]:
# query it
query = "Where do we live?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

My mother's name is Jenny. My uncle's name is Tim. We live in Avatar City, Dumbo Street, United States of Dickens.


In [21]:
# query it
query = "What is 1+1?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

My mother's name is Jenny. My uncle's name is Tim. We live in Avatar City, Dumbo Street, United States of Dickens.


## Faiss

[Facebook AI Similarity Search (Faiss)](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

[FAISS Doc](https://faiss.ai/)



This notebook shows how to use functionality related to the FAISS vector database.

In [26]:
# For CUDA 7.5+ Supported GPU's.
! pip install faiss-gpu
# OR
# pip install faiss-cpu # For CPU Installation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.9 MB/s eta 0:00:00


In [31]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [34]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [27]:
import getpass
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

### Basic Example

In [29]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [32]:
loader = TextLoader("some_file.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [35]:
# create FAISS database
db = FAISS.from_documents(docs, embeddings)

In [38]:
# question
query = "What's the mother's name?"
docs = db.similarity_search(query)

# answer
print(docs[0].page_content)

My mother's name is Jenny. 

        
        My uncle's name is Tim. 

        
        We live in Avatar City, Dumbo Street, United States of Dickens.


For more about langchain integrations of vector stores, please see [here](https://python.langchain.com/docs/integrations/vectorstores/chroma).